In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import joblib

# Load the dataset containing labeled fake news articles
# Load the true and false CSV files
true_data = pd.read_csv('data/Truenews.csv')
false_data = pd.read_csv('data/Fakenews.csv')

# Add a 'label' column to distinguish between true and false news
true_data['label'] = 1  # 1 for true news
false_data['label'] = 0  # 0 for false news

# Combine the two DataFrames into a single DataFrame
data = pd.concat([true_data, false_data], ignore_index=True)
data.head()



,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [2]:

# Ensure that the 'text' column contains only strings (not floats)
data = data.dropna(subset=['text'])
data['text'] = data['text'].astype(str)

# Text preprocessing function for fake news articles
def preprocess_news(text):
    # Customize text preprocessing here
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])
    
    # Tokenization (split text into words)
    tokens = word_tokenize(text)
    
    # Remove stopwords (common words like "the", "and", "is")
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Rejoin tokens into a single string
    text = ' '.join(tokens)
    
    return text

# Apply text preprocessing to the 'text' column
data['text'] = data['text'].apply(preprocess_news)

# Create a TF-IDF vectorizer to convert text data into feature vectors
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(data['text'].values)
y = data['label'].values
joblib.dump(vectorizer, 'newsTfidfVectorizer.sav')

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train the Random Forest classifier
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)
joblib.dump(random_forest, 'newsmodel.sav')
# Make predictions on the test set
rf_predictions = random_forest.predict(X_test)

# Calculate evaluation metrics for the Random Forest classifier
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)
rf_f1 = f1_score(y_test, rf_predictions)
rf_confusion = confusion_matrix(y_test, rf_predictions)

# Print evaluation metrics for the Random Forest classifier
print("Random Forest Metrics:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)
print("F1-Score:", rf_f1)
print("Confusion Matrix:")
print(rf_confusion)




Random Forest Metrics:
Accuracy: 0.9971046770601336
Precision: 0.9969976905311778
Recall: 0.9969976905311778
F1-Score: 0.9969976905311778
Confusion Matrix:
[[4637   13]
 [  13 4317]]


In [ ]:
# Generate word clouds
true_data1 = " ".join(true_data['text'])
false_data1 = " ".join(false_data['text'])
true_wordcloud = WordCloud(width=800, height=400).generate(true_data1)
false_wordcloud = WordCloud(width=800, height=400).generate(false_data1)


In [ ]:
# Visualize confusion matrix for the Random Forest classifier
plt.figure(figsize=(6, 6))
sns.heatmap(rf_confusion, annot=True, cmap='Blues', fmt='d', cbar=False)
plt.title('Confusion Matrix for Random Forest Classifier')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()

In [ ]:

# Plot the word clouds
plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.imshow(true_wordcloud, interpolation='bilinear')
plt.title('Word Cloud for True News')
plt.axis("off")

plt.subplot(122)
plt.imshow(false_wordcloud, interpolation='bilinear')
plt.title('Word Cloud for False News')
plt.axis("off")

plt.tight_layout()
plt.show()